In [2]:
import gym
import numpy as np
import random
from tqdm import tqdm

In [4]:
ENV_NAME = 'CartPole-v0'

env = gym.make(ENV_NAME)
print(f'action space: {env.action_space}')
print(f'state space: {env.observation_space}')

action space: Discrete(2)
state space: Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)


/home/cmaspi/.local/lib/python3.10/site-packages/gym/envs/registration.py:505: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1` with the environment ID `CartPole-v1`.
  logger.warn(


In [21]:
def random_agent(env):
    return np.random.choice(env.action_space.n, 1)[0]


def simulate(num_episodes: int, render=False):
    env = gym.make(ENV_NAME)
    scores = []
    for episode in range(1, num_episodes+1):
        total_score = 0
        state = env.reset()
        done = False
        itr = 0
        while not done:
            itr += 1
            if render:
                env.render()
            action = random_agent(env)
            next_state, reward, done, *_ = env.step(action)
            total_score += reward
        scores.append(total_score)
        print(itr)
    env.close()
    return scores


scores = simulate(10, True)
print(scores)


/home/cmaspi/.local/lib/python3.10/site-packages/gym/envs/registration.py:505: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1` with the environment ID `CartPole-v1`.
  logger.warn(


16
46
26
17
21
14
31
25
10
10
[16.0, 46.0, 26.0, 17.0, 21.0, 14.0, 31.0, 25.0, 10.0, 10.0]


The random agent performs poorly, the game stops within a very short time frame because it breaches one of the two mentioned conditions, that are

1. It shouldn't be off by more than 2.4 units
2. The angle from vertical shouldn't be more than 15 degrees.

A reward of +1 is given for each time stamp that the pole stays upright.